In [1]:
import csv
import os
import random
import re
import string
import unicodedata
import zlib
from collections import defaultdict
from itertools import combinations
from typing import Callable, Optional

import numpy as np
import pandas as pd
import spacy
from scipy.optimize import fsolve
from tqdm.auto import tqdm

In [2]:
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_sm')

In [ ]:
def is_prime(n: int):
    if n < 2:
        return False
    for i in range(2, int(np.sqrt(n))+1):
        if (n % i) == 0:
            return False
    return True

def find_closest_prime(n):
    while True:
        if is_prime(n):
            return n
        n += 1

class HashGenerator:
    def __init__(
        self, 
        num_rows: int = np.iinfo(np.uint32).max, 
        # prime: int = 4294967387
    ) -> None:
        # assert is_prime(prime)
        # assert prime >= num_rows

        self.num_rows = num_rows
        self.prime = find_closest_prime(num_rows)
        self.a_set = set()
        self.b_set = set()

    def get_num_rows(self) -> int:
        return self.num_rows

    def next(self) -> Callable[[np.uint32], np.uint32]:
        a = self._generate_coeff(self.a_set, self.num_rows)
        b = self._generate_coeff(self.b_set, self.num_rows)
        return lambda row: np.uint32((a * row + b) % self.prime)

    def reset(self) -> None:
        self.a_set = set()
        self.b_set = set()

    def _generate_coeff(
        self, 
        coeff_set: set[int],
        max_val: int
    ) -> int:
        while True:
            coeff = random.randint(1, max_val)
            if coeff not in coeff_set:
                coeff_set.add(coeff)
                return coeff
    

# Preprocessing

In [3]:
def normalize_white_space(doc: str) -> str:
    return " ".join(doc.split())

def remove_https(doc: str) -> str:
    return re.sub(r'https?://[^ ]+', '', doc)

def replace_chars(doc: str) -> str:
    doc = doc.replace('&amp;', ' and ')
    doc = doc.replace('@', ' ')
    doc = doc.replace('$', ' ')
    doc = doc.replace('_', ' ')
    doc = doc.replace('|', ' ')

    return doc

def remove_non_ascii(doc: str) -> str:
    """We keep cyrillic characters due to the nature
    of the dataset.
    """
    cyr_chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"

    res = ""
    for c in doc:
        if (c.isascii() and c.isprintable()) \
            or (c in cyr_chars) or c.isspace():
            res += c
    return res

def strip_accents(doc: str) -> str:
    """Deal with accents and special characters such as
    𝕒, 𝕕, 𝕖, 𝙖, 𝙘, 𝙙.
    """
    return unicodedata.normalize('NFKD', doc)

def strip_punctuation(doc: str) -> str:
    return re.sub('[' + re.escape(string.punctuation) + ']', '', doc)
    
def get_lemmatizer( 
    nlp: spacy.pipeline, 
    allow_stop_words: bool = False,
    allow_punct: bool = False,
    allow_numbers: bool = False
) -> Callable[[str], str]:
    def inner_f(doc):
        return ' '.join(
            [
                token.lemma_
                for token in nlp(doc)
                if (not token.is_stop or allow_stop_words) \
                    and (not token.is_punct or allow_punct) \
                    and (token.pos_ != 'NUM' or allow_numbers) \
                    and (not token.pos_ == 'X')
            ]
        )
    return inner_f

In [ ]:
def jaccard_similarity(
    x: np.ndarray, 
    y: np.ndarray
) -> float:
    numerator = len(set(x).intersection(set(y)))
    denominator = len(set(x).union(set(y)))
    return numerator / denominator

class LSHModel:
    def __init__(
        self,
        k: int,
        threshold: float,
        num_hashes: int,
        hash_generator: HashGenerator,
        shingle_hash: Callable = zlib.crc32
    ) -> None:
        self.k = k
        self.threshold = threshold
        self.num_hashes = num_hashes
        self.hash_generator = hash_generator
        self.num_rows = self.hash_generator.get_num_rows()
        self.shingle_dict = dict()
        # self.num_shingles = 0
        self.shingle_hash = shingle_hash
        self.num_docs = 0
        self.docs_dict = dict()
        self.signature = None
        self.candidate_pairs = set()
        self.fp_pairs = set()
        self.similar_pairs = set()
        self.b = -1
        self.r = -1

    def add_document(
        self, 
        doc: str,
        preprocessing_pipeline: Optional[list[Callable[[str], str]]] = None
    ) -> None:
        if preprocessing_pipeline is not None:
            for f in preprocessing_pipeline:
                doc = f(doc)
        shingles = self._create_shingles(doc, self.k)
        self.docs_dict[self.num_docs] = shingles
        self.num_docs += 1

    def get_similar_pairs(self) -> set[tuple[int, int]]:
        if self.num_hashes > len(self.shingle_dict):
            raise ValueError(
                f"Number of hash functions must be lower than "
                f"or equal to the number of shingles. Found "
                f"{self.num_hashes} hash functions and "
                f"{len(self.shingle_dict)} shingles."
            )
        
        hg = HashGenerator(len(self.shingle_dict))
        hash_functions = [
            hg.next()
            for _ in range(self.num_hashes)
        ]
        self.signature = self._build_signature(
            self.docs_dict,
            len(self.shingle_dict),
            hash_functions
        )
        self.b, self.r = self._find_lsh_params(
            self.threshold,
            self.num_hashes
        )
        self.candidate_pairs = self._lsh(
            self.signature,
            self.b
        )
        self.similar_pairs, self.fp_pairs = \
            self._check_threshold_on_signature(
                self.candidate_pairs,
                self.signature,
                self.threshold
            )
        return self.similar_pairs
        
    def _create_shingles(
        self, 
        doc: str,
        k: int
    ) -> np.ndarray:
        encoded_shingles = []
        
        for i in range(len(doc[:-k+1])):
            shingle = doc[i:i+k]
            if not shingle in self.shingle_dict:
                self.shingle_dict[shingle] = \
                    len(self.shingle_dict)

            encoded_shingles.append(
                self.shingle_dict[shingle]
            )

        return np.unique(encoded_shingles).astype(np.uint32)

    def _build_signature(
        self,
        docs_dict: dict[int, np.ndarray],
        num_rows: int, 
        hash_functions: list[Callable[[np.uint32], np.uint32]]
    ) -> np.ndarray:
        signature = np.full(
            (len(hash_functions), len(docs_dict)), 
            fill_value=np.inf
        )
        for r in tqdm(range(0, num_rows)):
            hash_values = [
                f(r)
                for f in hash_functions
            ]
            for c, shingles in enumerate(docs_dict.values()):
                if r in shingles:
                    for i, hash_val in enumerate(hash_values):
                        if hash_val < signature[i,c]:
                            signature[i,c] = hash_val 

        return signature.astype(np.uint32)

    def _find_lsh_params(self, t: int, n: int) -> tuple[int]:
        """A lower b means that two items must match a higher
        number of rows. By taking the floor of b, we favor
        more similar pairs.  
        """
        def equations(vars):
            b, r = vars
            eq1 = t - (1 / b) ** (1 / r)
            eq2 = n - b * r
            return [eq1, eq2]

        b, r =  fsolve(equations, (1, 1))
        b = np.floor(b)
        r = n // b
        return int(b), int(r)

    def _lsh(
        self, 
        signature: np.ndarray, 
        b: int
    ) -> set[tuple[int, int]]:
        candidate_pairs = set()
        
        for band in np.array_split(signature, b):
            
            # column tuple -> list of column indices having that tuple
            same_columns = defaultdict(list) 
            
            for c in range(band.shape[1]):
                column = band[:,c]
                same_columns[tuple(column)].append(c)

            filtered_same_columns = dict()
            for k, values in same_columns.items():
                if len(values) >= 2:
                    filtered_same_columns[k] = values

            for values in filtered_same_columns.values():
                for pair in combinations(values, 2):
                    candidate_pairs.add(pair)

        return candidate_pairs

    def _check_threshold_on_signature(
        self, 
        candidate_pairs: list[tuple[int, int]], 
        signature: np.ndarray, 
        t: float
    ) -> tuple[set[tuple[int, int, float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in candidate_pairs:
            x_col = signature[:,x]
            y_col = signature[:,y]
            similarity = sum(x_col == y_col) / signature.shape[0]
            tup = (x, y, similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

    def check_threshold_on_cm(
        self,
        candidate_pairs: list[tuple[int, int]], 
        docs_dict: dict[int, np.ndarray], 
        t: float
    ) -> tuple[set[tuple[int, int, float]]]:
        similar_pairs = set()
        false_positive_pairs = set()

        for (x, y) in candidate_pairs:
            similarity = jaccard_similarity(docs_dict[x], docs_dict[y])
            tup = (x, y, similarity)
            if similarity >= t:
                similar_pairs.add(tup)
            else:
                false_positive_pairs.add(tup)

        return similar_pairs, false_positive_pairs

# Test

In [4]:
dataset_path = r"E:\datasets\ukraine"

In [5]:
files = []

for name in os.listdir(dataset_path):
    full_path = os.path.join(dataset_path, name)
    if os.path.isfile(full_path):
        files.append(full_path)

df = pd.read_csv(
    files[0], 
    compression='gzip', 
    index_col=0,
    encoding='utf-8', 
    quoting=csv.QUOTE_ALL
)

d:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\IPython\core\interactiveshell.py:3012: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df = df[df['language'] == 'en']

In [11]:
char_set = set()

In [12]:
pipeline = [
        str.lower,
        remove_https,
        replace_chars,
        strip_accents,
        normalize_white_space,
        get_lemmatizer(nlp),
        strip_punctuation,
        remove_non_ascii,
    ]

for index, row in tqdm(
    df.iterrows(),
    total=len(df),
):
    text = row['text']
    for f in pipeline:
        text = f(text)
    text_set = set(text)
    print(text)
    # print(row['text'])
    char_set = char_set.union(text_set)
    if index > 2000:
        break

  0%|          | 13/254626 [00:00<1:14:38, 56.85it/s]

 ukrainian air force like address misinformation publish multiple western medium outlet situation  sky support nato ally  protectuаsky stoprussia ukraineunderattaсk
chernihiv oblast ukrainian welcome liberator russianinvasion ukraineunderattack ukrainewillwin putinisawarcriminal stopputin russianukrainianwar russiagohome россиясмотри нетвоине
america   prepare bad russianukrainianwar taiwan  month enter new cold war russia  good policy future   relation taiwan video  taiwan
anonymous hack release email marathon group russian investment firm own oligarch alexander vinokurov currently eu sanction vinokurov son law russian foreign minister lavrov oprussia ddosecret
public mint live billionaire wmn win public sale mint visit mint information good luck nft mint
amazing story sussex county delaware councilman vance phillips extremely difficult successful enterprise get ukrainian wife united states russia ukraine motivation netde edude delaware government usa 
quothow wait quot chernihiv regi

  0%|          | 40/254626 [00:00<41:44, 101.66it/s] 

putin declare unfriendly country open rouble account russian bank buy natural gas start april 1st
jaime antoine discuss initial reaction ukraine crisis ukraine ukrainerussianwar google
govkathyhochul love covid update hunter biden neerajkhemlani hack biden covid politics vote chickendanco retr0mikey
vladimirputin face mutiny russian president lose control soldier ditch uniform refuse order fight force
iaea director general rafaelmgrossi arrive kaliningrad russia talk senior russian official tomorrow morning
world focus ukrainewar family tigray ethiopia face 500daysoftigraygenocide remain forget ic accord wfp child pregnant breastfeed woman malnourish unocha
ukraine claim withdraw vdv convoy kyiv oblast attack ukrainian force airborne infantry fighting vehicle completely destroy extremely gruesome scene
air war ukraine month old episode chainreaction discuss airwar role defense play frustrating moscow campaign tune 
ukraine ua force capture russian t90a tank good condition unaware moder

  0%|          | 66/254626 [00:00<37:02, 114.53it/s]

...

ukrainian armed force liberate settlement kherson region seize trophy equipment
brussel sunday la monnaie de munt go light candle victim russian war ukraine standwithukriane stopputinnow helpukraine mariupol chernihiv belgium belgique bruxelle
good idea toomas ilvestoomas long russia deport ukrainian occupied territory russians allow eu period putinwarcrime putinwarcriminal
ukraine russian fall george friedman george friedman gpfutures founder chairman geopolitical future audioboom ukraine
bambam spot incheon international airport way usa 29th march camila cabello ed sheeran join force itv concert ukraine perform new collaboration bam bam live time bambamtousa2022 bambam b bambam1a
break company russia non friendly country need open ruble account russian bank purchase gas country start april accord decree sign president vladimir putin
year ago ukraine
finland putin blood oil head way reject standwithukraine 
stray dog accompany ukrainian fighter odin unit take cover inside building hea

  1%|          | 1604/254626 [00:12<33:29, 125.93it/s]

report irpin today cnn ukraine russia
heart break image people stand line bread child suffer hospital family separation city turn ash street line dead american support putin lack empathy possibly soul ukraine
ukrainian theater war day today update briefly focus chernihiv sumy oblast ukrainian force important gain sumy russia prepare long term defense chernihiv oblast ukrainerussianwar ukraine russianukrainianwar
win war stand  ukraine ukrainerussiawar mint favourite peace nightingale 
 ukrainian air force like address misinformation publish multiple western medium outlet situation  sky support nato ally  protectuаsky stoprussia ukraineunderattaсk
ukraine destroy russian kamaz63968 typhoon k mrap kyiv oblast
tweet week ago putin new plan annex kharkiv donbas southern coast russia leave ukraine troop simply regroup return soon concentrate area
russia say ukraine crisis reach arctic nato say
ander aslund putin respect human life
 ukrainian air force like address misinformation publish mul

  1%|          | 1624/254626 [00:12<33:14, 126.86it/s]

note week evidence syrian ukraine evidence organize contractual recruitment syrian ukraine evidence russia remove heavy weaponry libya syria ukraine unproven biased social medium claim   evidence
russia point real threat wmds evidence public biolabsinukraine washington regime chemical biological nuclear civilian population
world focus ukraine urge un member forget tigray region cut world imperative reject ethiopia effort deny fund investigation voteno tigraycantwait thailandun
good idea toomas ilvestoomas long russia deport ukrainian occupied territory russians allow eu period putinwarcrime putinwarcriminal
poll bilibili chinese equivalent youtube ask blame russia ukraine conflict user vote winner iiiiiiiiiiiiis wait guess course thegreattranslationmovement  ukrainerussiawar
ukrainian artillery target russian military hardware station residential building russia ukraine
russia present new evidence ukraine biolab comment link biden rt russia soviet union biolab ukraine biden
  japanese 

## check

In [13]:
len(char_set)

53

In [14]:
char_set

{' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'а',
 'в',
 'г',
 'д',
 'е',
 'и',
 'л',
 'м',
 'н',
 'о',
 'р',
 'с',
 'т',
 'ц',
 'ь',
 'я'}

In [ ]:
len(char_set)

In [ ]:
'誌'.isalnum()

In [ ]:
char_set

In [ ]:
df[0:2].T

# Test Small

In [ ]:
hg = HashGenerator()
docs = [
    "Lincoln was born into poverty in a log cabin in Kentucky and was raised on the frontier, primarily in Indiana. He was self-educated and became a lawyer, Whig Party leader, Illinois state legislator, and U.S. Congressman from Illinois. In 1849, he returned to his law practice but became vexed by the opening of additional lands to slavery as a result of the Kansas–Nebraska Act of 1854. He reentered politics in 1854, becoming a leader in the new Republican Party, and he reached a national audience in the 1858 Senate campaign debates against Stephen Douglas. Lincoln ran for President in 1860, sweeping the North to gain victory. Pro-slavery elements in the South viewed his election as a threat to slavery, and Southern states began seceding from the Union. During this time the newly formed Confederate States of America began seizing federal military bases in the south. Just over one month after Lincoln assumed the presidency, the Confederate States attacked Fort Sumter, a U.S. fort in South Carolina. Following the bombardment, Lincoln mobilized forces to suppress the rebellion and restore the Union.",
    "Abraham Lincoln was born on February 12, 1809, the second child of Thomas Lincoln and Nancy Hanks Lincoln, in a log cabin on Sinking Spring Farm near Hodgenville, Kentucky.[2] He was a descendant of Samuel Lincoln, an Englishman who migrated from Hingham, Norfolk, to its namesake, Hingham, Massachusetts, in 1638. The family then migrated west, passing through New Jersey, Pennsylvania, and Virginia.[3] Lincoln's paternal grandparents, his namesake Captain Abraham Lincoln and wife Bathsheba (née Herring) moved the family from Virginia to Jefferson County, Kentucky.[b] The captain was killed in an Indian raid in 1786.[5] His children, including eight-year-old Thomas, Abraham's father, witnessed the attack.[6][c] Thomas then worked at odd jobs in Kentucky and Tennessee before the family settled in Hardin County, Kentucky, in the early 1800s.",
    "A supernova is a powerful and luminous stellar explosion. This transient astronomical event occurs during the last evolutionary stages of a massive star or when a white dwarf is triggered into runaway nuclear fusion. The original object, called the progenitor, either collapses to a neutron star or black hole, or is completely destroyed. The peak optical luminosity of a supernova can be comparable to that of an entire galaxy before fading over several weeks or months. Supernovae are more energetic than novae. In Latin, nova means new, referring astronomically to what appears to be a temporary new bright star. Adding the prefix super- distinguishes supernovae from ordinary novae, which are far less luminous. The word supernova was coined by Walter Baade and Fritz Zwicky in 1929.",
    "The most recent directly observed supernova in the Milky Way was Kepler's Supernova in 1604, but the remnants of more recent supernovae have been found. Observations of supernovae in other galaxies suggest they occur in the Milky Way on average about three times every century. These supernovae would almost certainly be observable with modern astronomical telescopes. The most recent naked-eye supernova was SN 1987A, the explosion of a blue supergiant star in the Large Magellanic Cloud, a satellite of the Milky Way."
]

In [ ]:
hg = HashGenerator()
docs = [
    "The    pen is     on   the       table",
    "The cat is eating something on the table",
    "I watched soccer on television"
]

In [ ]:
model = LSHModel(
    k=5,
    threshold=0.1,
    num_hashes=10,
    hash_generator=hg
)

In [ ]:
preprocessing_pipeline = [
    remove_https,
    normalize_white_space,
    get_lemmatizer(nlp)
]

In [ ]:
for doc in docs:
    model.add_document(doc, preprocessing_pipeline)

In [ ]:
model.get_similar_pairs()

In [ ]:
model.check_threshold_on_cm(
    model.candidate_pairs,
    model.docs_dict,
    model.threshold
)

In [ ]:
model.docs_dict

In [ ]:
model.signature

In [ ]:
2 ** 32

In [ ]:
2 ** 24

In [ ]:
960 ** 5

In [ ]:
ss = "#standwithukraine️ #ukraineunderattack #ukrainewillwin #putinisawarcriminal #stopputin #russianukrainianwar #russiagohome #россиясмотри #нетвойне https://t.co/86ixyuetnb america 🇺🇸 is preparing for something worse than the #russianukrainianwar...taiwan 🇹🇼 last month we entered into a new cold war with russia 🇷🇺 what's the best policy for the future of 🇺🇸🇨🇳 relations over taiwan?"

In [ ]:
remove_https(ss)

In [ ]:
ss2 = "'!''#'' biden ''(biden)"

In [ ]:
strip_punctuation2(ss2)

In [ ]:
def strip_punctuation(doc: str) -> str:
    return doc.replace_chars(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')